### Imports

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from geopy.geocoders import Nominatim 
geolocator = Nominatim(user_agent="geoapiExercises")
import pickle


### Download the dataset from paleobiodb.org

1. Create an account on paleobiodb.org
2. Go to the downloads page
3. Set some parameters to generate your download URL
4. What do you want to download? Specimens
5. Format: Comma-separated values
6. Select by metadata -> check "Select all specimen records in the database"
7. Choose output options -> check only "classification" and "coordinates"
8. Scroll up to the "Test" and "Download" buttons and you will see the generated URL
9. Click the download button
10. Copy it to your project's data folder and rename it pbdb_data_world.csv

### Shrink the dataset
The original download is too large (~2.15GB).  It contains many fields that we don't need and many rows that we cannot use because they don't have values for latitude or longitude, etc.

Also, this dataset is too large to store in GitHub (which has a 100 MB filesize limit).

So, in this step, we'll shrink it down to the dataset that we need.

In [ ]:
# Read in the original large dataset
df = pd.read_csv('../data/pbdb_data_world.csv')

# Limit the data to just what we need
df = df[['genus','accepted_name','lat','lng','state','county']]

# Drop any rows that contain NaN values
df = df.dropna()

# Save the new file
df.to_csv('../data/name_and_location.csv')


### Import the data

In [ ]:
df = pd.read_csv('../data/name_and_location.csv')


### Explore the data

In [ ]:
## How many records are there?
print('df.shape: {}'.format(df.shape))

## How many unique latitude and longitude pairs are there in the records?
column_values = df[['lat','lng']].values.ravel()
unique_lat_lng_values =  pd.unique(column_values)
print('len(unique_lat_lng_values): {}'.format(len(unique_lat_lng_values)))


### Decide to implement a two-step process

The dataframe is missing a column for country.  I'd like that "country" column so I can easily filter the dataframe by country, state/province, and county.  I can calculate it from the latitude and longitude using the geolocation features of the geopy library.  However, labeling each of these records with a "country" will be a fair amount of work for the program.

Given that the number of unique latitude/longitude pairs is around 1/10th of the number of total records, it would be much better to create a dictionary with a lat/lng key and a country value for those unique lat/lng pairs.  Then, as a second step, use that dictionary to populate a new "country" column in the dataframe.  The dictionary lookups will be much faster because the dictionary object uses a HashTable algorithm.

### Step 1: Build the country lookup dictionary

In [ ]:
# Create a function that will call the geolocator library to return the country
def getCountry(lat, lng):
    country = ''
    try:
        location = geolocator.reverse(str(round(lat, 2))+","+str(round(lng))) 
        address = location.raw['address'] 
        country = address.get('country', '') 
    except:
        country = ''
    finally:
        return country

# Use the function to label each unique lat/lng combination with its country
country_lookup = {}
counter = 0
while counter < unique_values.size:
    lat = unique_values[counter]
    lng = unique_values[counter + 1]
    country_lookup[str(lat) + ',' + str(lng)] = getCountry(lat, lng)
    if counter % 5000 == 0:
        print('counter = {}'.format(counter))
    counter += 2

# This took about 12 hours so, to make it easier to load in the future without
# redoing all this work, I'm going to save this dictionary.
pickle.dump( country_lookup, open( "../data/country_lookup.p", "wb" ) )


### Use the country_lookup dictionary to populate a new "country" column

In [ ]:
def lookupCountry(row):
    lat, lng = row['lat'], row['lng']
    try: 
        country = country_lookup[str(lat) + ',' + str(lng)] 
        return country
    except:
        return ''
    
df['country'] = df.apply(lookupCountry, axis=1)

# Save the dataframe (with the new country column) for easy loading later
df.to_csv('../data/name_and_location_with_country.csv')
